# Colab setting

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/Othercomputers/MacBook/Earth/module/dd_earthquake/book

Mounted at /content/drive
/content/drive/Othercomputers/MacBook/Earth/module/dd_earthquake/book


In [3]:
%ls

bench_mark.ipynb  colab_lgbt_multigrid.ipynb     Light_GBM.ipynb
catboost_info/    edm_data.ipynb
Catboost.ipynb    learn_categorical_plots.ipynb


Install lgbm with gpu option

ref: https://an-engineer-note.com/?p=624

In [ ]:
!pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm

Found existing installation: lightgbm 2.2.3
Uninstalling lightgbm-2.2.3:
  Successfully uninstalled lightgbm-2.2.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Skipping wheel build for lightgbm, due to binaries being disabled for it.


# lgbm

In [7]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import numpy as np
import pandas as pd
import warnings
import pprint

import lightgbm as lgb
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from src import common

## grid search

In [61]:
train_values, train_labels = common.file.read_data('train')

In [7]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,0
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0


In [62]:
features_list = ['geo_level_1_id', 
                 'geo_level_2_id', 
                 'geo_level_3_id',
                 'age', 
                 'area_percentage', 
                 'height_percentage', 
                 'foundation_type', 
                 'roof_type', 
                 'ground_floor_type', 
                 'other_floor_type', 
                 'position', 
                 'has_superstructure_mud_mortar_stone', 
                 'has_superstructure_cement_mortar_brick', 
                 'has_superstructure_timber', 
                 'count_families', 
                 'has_secondary_use'
                ]

In [63]:
train_values, train_labels = common.lgbm_preprocessing((train_values, train_labels), mode='training', features_list=features_list)

In [64]:
x_train, x_test, y_train, y_test = train_test_split(train_values, train_labels,
                                                        test_size=0.1, random_state=19, stratify=train_labels)

In [11]:
skf = StratifiedKFold(n_splits=5,
                      shuffle=True,
                      random_state=0)

In [24]:
# initialize model and parameter grid
model = lgb.LGBMClassifier(objective='multi_class', num_class=3, 
                           learning_rate=0.15, 
                           force_row_wise=True, 
                           num_boost_round=200, 
                           device='gpu', 
                           # valid_sets=valids
                           )

param_grid = {
    "max_depth": [10, 25, 50, 75],
    # "learning_rate" : [0.001,0.01,0.05,0.1],
    # "num_leaves": [100,300,900,1200],
    "n_estimators": [100,200,500], 
    # 'reg_alpha': list(map(lambda x: x*0.1, range(1, 5))), 
    # 'reg_lambda': [0.1, 0.2],
             }

# grid search
grid_result = GridSearchCV(estimator = model,
                           param_grid = param_grid,
                           scoring = 'f1_micro', 
                           cv = skf,
                           return_train_score = True,
                           n_jobs = -1)

grid_result.fit(x_train, np.reshape(y_train, -1))

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=200, num_boost_round=200 will be ignored. Current value: num_iterations=200


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=LGBMClassifier(device='gpu', force_row_wise=True,
                                      learning_rate=0.15, num_boost_round=200,
                                      num_class=3, objective='multi_class'),
             n_jobs=-1,
             param_grid={'max_depth': [10, 25, 50, 75],
                         'n_estimators': [100, 200, 500]},
             return_train_score=True, scoring='f1_micro')

In [25]:
pprint.pprint(grid_result.best_estimator_)

LGBMClassifier(device='gpu', force_row_wise=True, learning_rate=0.15,
               max_depth=75, n_estimators=500, num_boost_round=200, num_class=3,
               objective='multi_class')


In [26]:
pprint.pprint(grid_result.best_score_)

0.7287754753986526


# Use best estimator 

In [8]:
train_values, train_labels = common.file.read_data('train')

In [9]:
features_list = ['geo_level_1_id', 
                 'geo_level_2_id', 
                 'geo_level_3_id',
                 'age', 
                 'area_percentage', 
                 'height_percentage', 
                 'foundation_type', 
                 'roof_type', 
                 'ground_floor_type', 
                 'other_floor_type', 
                 'position', 
                 'has_superstructure_mud_mortar_stone', 
                 'has_superstructure_cement_mortar_brick', 
                 'has_superstructure_timber', 
                 'count_families', 
                 'has_secondary_use'
                ]

In [10]:
train_values, train_labels = common.lgbm_preprocessing((train_values, train_labels), mode='training', features_list=features_list)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(train_values, train_labels,
                                                        test_size=0.1, random_state=19, stratify=train_labels)

In [12]:
trains = lgb.Dataset(x_train, y_train)
valids = lgb.Dataset(x_test, y_test)

In [16]:
params = {
    "objective": "multiclass",
    "num_class": 3, 
    "metrics": "multi_logloss",
    'device': 'gpu', 
    'max_depth': 75, 
    'force_row_wise': True, 
    "learning_rate": 0.05, 
    'reg_alpha': 0.3,
    'reg_lambda': 0.3,
}

In [17]:
model = lgb.train(params, trains, valid_sets=valids, num_boost_round=20000, early_stopping_rounds=1000)

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


Streaming output truncated to the last 5000 lines.
[86]	valid_0's multi_logloss: 0.665413
[87]	valid_0's multi_logloss: 0.665027
[88]	valid_0's multi_logloss: 0.664725
[89]	valid_0's multi_logloss: 0.664346
[90]	valid_0's multi_logloss: 0.663815
[91]	valid_0's multi_logloss: 0.663457
[92]	valid_0's multi_logloss: 0.66309
[93]	valid_0's multi_logloss: 0.662702
[94]	valid_0's multi_logloss: 0.662283
[95]	valid_0's multi_logloss: 0.661894
[96]	valid_0's multi_logloss: 0.661608
[97]	valid_0's multi_logloss: 0.661283
[98]	valid_0's multi_logloss: 0.660882
[99]	valid_0's multi_logloss: 0.660517
[100]	valid_0's multi_logloss: 0.660236
[101]	valid_0's multi_logloss: 0.659953
[102]	valid_0's multi_logloss: 0.659691
[103]	valid_0's multi_logloss: 0.659399
[104]	valid_0's multi_logloss: 0.659159
[105]	valid_0's multi_logloss: 0.658818
[106]	valid_0's multi_logloss: 0.658576
[107]	valid_0's multi_logloss: 0.658179
[108]	valid_0's multi_logloss: 0.657758
[109]	valid_0's multi_logloss: 0.65748
[110]

In [19]:
# predict

# テストデータの予測 ((各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率] を返す))
y_pred_prob = model.predict(x_test)
# テストデータの予測 (予測クラス(0 or 1 or 2)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
# 真値と予測値の表示
df_pred = pd.DataFrame({'target':y_test['damage_grade'].values,'target_pred':y_pred})
display(df_pred)

# 真値と予測確率の表示
df_pred_prob = pd.DataFrame({'y':y_test['damage_grade'].values, 'target0_prob':y_pred_prob[:,0], 'target1_prob':y_pred_prob[:,1], 'target2_prob':y_pred_prob[:,2]})
display(df_pred_prob)

acc = f1_score(y_test,y_pred, average='micro')
print('Acc :', acc)

,target,target_pred
0,1,1
1,2,2
2,1,2
3,1,1
4,1,1
...,...,...
26056,0,1
26057,1,1
26058,1,1
26059,0,1


,y,target0_prob,target1_prob,target2_prob
0,1,0.042023,0.622313,0.335664
1,2,0.002117,0.454850,0.543033
2,1,0.000342,0.399571,0.600088
3,1,0.001897,0.894754,0.103348
4,1,0.220040,0.709849,0.070111
...,...,...,...,...
26056,0,0.452068,0.494931,0.053000
26057,1,0.201995,0.573719,0.224286
26058,1,0.349016,0.605332,0.045652
26059,0,0.257338,0.699821,0.042841


Acc : 0.7465177851962704


# submit

In [20]:
test_values = common.file.read_data('test')

In [21]:
test_values, _ = common.lgbm_preprocessing(test_values, mode='test', features_list=features_list)

In [22]:
y_test_prob = model.predict(test_values)
y_test = np.argmax(y_test_prob, axis=1)

In [23]:
submission_format = common.file.read_data('submission')
my_submission = pd.DataFrame(data=y_test+1,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [24]:
my_submission.to_csv('../../../data/final/submit/gridsearch_20000trial.csv')